# LDA para encontrar dialectos?

Vamos a intentar un poquito de esto...

In [1]:
from pymongo import MongoClient

client = MongoClient('localhost')

db = client['contrastes']

Vamos a hacer lo siguiente: usemos cada usuario como "documento"

In [2]:
db.tweets.find_one()

{'_id': ObjectId('5ba53c9827a5141aaa383eb9'),
 'created_at': 'Tue Nov 05 14:48:51 +0000 2013',
 'id': 397737276736040960,
 'place': None,
 'provincia': 'larioja',
 'text': 'Estoy tan asustada :(',
 'user_id': 301800629}

In [3]:
import numpy as np


user_ids = db.tweets.distinct("user_id")

user_ids = np.random.choice(user_ids, 600)
user_tweets = {}

for uid in user_ids:
    user_tweets[uid] = [t['text'] for t in db.tweets.find({'user_id': int(uid)})]


In [4]:
import seaborn as sns

sns.distplot([len(tweets) for _, tweets in user_tweets.items()], bins=100, kde=False)

Hay un montón de poquísimos tweets 🤦 veamos qué hacer...

bah, tampoco son tantos!

In [5]:
lens = np.array([len(tweets) for _, tweets in user_tweets.items()])

(lens <= 10).sum()

15

In [6]:
import pandas as pd
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
docs = {}

for uid, tweets in user_tweets.items():
    docs[uid] = [token for t in tweets for token in tokenizer.tokenize(t) if token.isalpha()]

#docs = {uid: ". ".join(tweets) for uid, tweets in user_tweets.items()}

Saquemos stop words, urls, y otras porquerías

In [7]:
# NLTK Stop words
import re

from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')

stop_words.extend([
    'x', 'xq', 'si', "q", "k", "d", "re"
])
url_re = 'https?://.*'
laugh_regex = '(a(a|j)*j(a|j)*)|(j(a|j)*a(a|j)*)'

def remove_with(filter_predicate, doc):
    return list(filter(filter_predicate, doc))

def remove_stops(doc):
    return remove_with(lambda tok: tok not in stop_words, doc)

def remove_urls(doc):
    return remove_with(lambda tok: not re.match(url_re, tok), doc)

def remove_non_alpha(doc):
    return remove_with(lambda t: t.isalpha(), doc)

def replace_lol(tok):
    if re.match(laugh_regex, tok):
        return "ja"
    else:
        return tok

def remove_lol(doc):
    return [tok for tok in doc if not re.match(laugh_regex, tok)]

docs_nostops = {uid: remove_lol(remove_stops(doc)) for uid, doc in docs.items()}

In [11]:
import gensim
import gensim.corpora as corpora
# Create Dictionary

texts = list(docs_nostops.values())

id2word = corpora.Dictionary(texts)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


# Human readable format of corpus (term-frequency)
[(id2word[id], freq) for id, freq in corpus[0]]

[('aa', 1),
 ('abajo', 1),
 ('abandonada', 1),
 ('abandoné', 1),
 ('abrazo', 3),
 ('abregu', 1),
 ('abri', 1),
 ('abrir', 1),
 ('abstinencia', 1),
 ('abuelo', 6),
 ('aburrida', 1),
 ('aburrimiento', 2),
 ('aburrirme', 1),
 ('aburro', 1),
 ('aburrooo', 1),
 ('aca', 3),
 ('acaba', 46),
 ('acabaram', 50),
 ('academia', 2),
 ('acessaram', 47),
 ('acessos', 54),
 ('acompañado', 1),
 ('acompañar', 1),
 ('acompañas', 1),
 ('acordar', 2),
 ('acordas', 2),
 ('acordate', 3),
 ('acorde', 1),
 ('acordes', 1),
 ('acostado', 5),
 ('acostando', 2),
 ('acoste', 1),
 ('acostumbrarse', 2),
 ('actores', 1),
 ('acuerde', 1),
 ('acuerdo', 6),
 ('acuerdooo', 1),
 ('acuesto', 1),
 ('acusticos', 1),
 ('acá', 2),
 ('adelante', 3),
 ('ademas', 1),
 ('además', 1),
 ('adentro', 1),
 ('adhiera', 1),
 ('adios', 1),
 ('admiro', 1),
 ('afana', 1),
 ('afanan', 1),
 ('afanar', 1),
 ('afanaron', 3),
 ('afano', 2),
 ('agarra', 1),
 ('agarrar', 1),
 ('agarre', 2),
 ('agora', 84),
 ('agosto', 2),
 ('agotadisimo', 1),
 ('ag

In [9]:
%%time
# Build LDA model

lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word)

CPU times: user 32.1 s, sys: 22.7 s, total: 54.8 s
Wall time: 23.6 s


In [10]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(19,
  '0.007*"xd" + 0.006*"voy" + 0.006*"hoy" + 0.005*"quiero" + 0.005*"vos" + '
  '0.005*"día" + 0.005*"mañana" + 0.004*"siempre" + 0.004*"bien" + '
  '0.004*"mas"'),
 (43,
  '0.007*"voy" + 0.006*"quiero" + 0.006*"hoy" + 0.005*"bien" + 0.005*"mas" + '
  '0.005*"vos" + 0.005*"amo" + 0.004*"día" + 0.004*"hace" + 0.004*"mañana"'),
 (81,
  '0.007*"quiero" + 0.007*"mas" + 0.007*"voy" + 0.006*"hoy" + 0.005*"amo" + '
  '0.005*"vos" + 0.005*"mañana" + 0.004*"siempre" + 0.004*"ganas" + '
  '0.004*"hace"'),
 (70,
  '0.007*"voy" + 0.006*"quiero" + 0.005*"mas" + 0.004*"vos" + 0.004*"tan" + '
  '0.004*"ganas" + 0.004*"hoy" + 0.004*"ser" + 0.004*"día" + 0.004*"mal"'),
 (14,
  '0.011*"perfil" + 0.010*"pessoas" + 0.009*"quiero" + 0.008*"seu" + '
  '0.007*"mas" + 0.007*"voy" + 0.005*"vos" + 0.005*"hoy" + 0.005*"ganas" + '
  '0.004*"visto"'),
 (55,
  '0.009*"quiero" + 0.008*"voy" + 0.008*"mas" + 0.006*"vos" + 0.005*"bien" + '
  '0.004*"hoy" + 0.004*"perfil" + 0.004*"va" + 0.004*"hace" + 0.004*"siempr